In [1]:
import pandas as pd
import time
from tqdm import tqdm
import pickle
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

# Spotify Data Collection

## 1. Establish credentials

In [2]:
#spotify API credentials

CLIENT_ID = "3d5bcb02c57e4f9e8404a1f736a87c95"
CLIENT_SECRET = "c956eeca7c25425d90ec0c0b87bc4e67"

credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

token = credentials.get_access_token()
sp = spotipy.Spotify(auth=token)

/Users/Vittorio/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  # Remove the CWD from sys.path while we load stuff.


## 2. Import and format data from Pitchfork

In [3]:
#import dataframe from pitchfork web scraping
pitchfork_df = pd.read_csv('data/albums_clean.csv')
pitchfork_df.drop(columns=['Unnamed: 0','url','label','year','reviewid','author','pub_year','content'], inplace=True)
pitchfork_df

,genre,artist,title,score
0,electronic,massive attack,mezzanine,9.3
1,metal,krallice,prelapsarian,7.9
2,rock,uranium club,all of them naturals,7.3
3,rock,kleenex,first songs,9.0
4,electronic,taso,new start,8.1
...,...,...,...,...
15161,rock,poster children,new world record,8.1
15162,rock,built to spill,keep it like a secret,9.3
15163,rock,califone,califone,9.2
15164,electronic,autechre,peel session,7.7


In [4]:
#creat list of lists for album-artist pairs in order to search spotify
alb_art_pairs = [[pitchfork_df.title[x], pitchfork_df.artist[x]] for x in range(len(pitchfork_df))]

In [5]:
len(alb_art_pairs)

15166

In [6]:
alb_art_pairs[0:5]

[['mezzanine', 'massive attack'],
 ['prelapsarian', 'krallice'],
 ['all of them naturals', 'uranium club'],
 ['first songs', 'kleenex'],
 ['new start', 'taso']]

## 3. Gather Spotify data about albums

In [7]:
#function for getting spotify album ID's from album name and artist pair
def GetAlbumID(album, artist):
    results = sp.search(q = "album:" + album + ' artist:' + artist, type = "album")
    return results['albums']['items'][0]['uri']

In [8]:
#test function
a = GetAlbumID('A Moon Shaped Pool', "Radiohead")
a

'spotify:album:6vuykQgDLUCiZ7YggIpLM9'

In [9]:
album_IDs2_combo = []
album_IDs2 = []

In [20]:
#get spotify albums' ID's
for pair in tqdm(alb_art_pairs):
    time.sleep(.25)
    try:
        album_IDs2_combo.append([GetAlbumID(pair[0],pair[1]),pair[0],pair[1]])
        album_IDs2.append([GetAlbumID(pair[0],pair[1])])
    except:
        album_IDs2_combo.append(['Spotify features Not found',pair[0],pair[1]])
        album_IDs2.append(['Spotify features Not found'])  

  0%|          | 22/15166 [00:21<3:30:30,  1.20it/s]

KeyboardInterrupt: 

In [10]:
#function for getting the track ID's for all of the songs in an album
def GetTrackIDs(album_id):
    track_id_list = []
    track_ids = sp.album_tracks(album_id)
    for track in track_ids['items']:
        track_id_list.append(track['id'])
    return track_id_list

In [11]:
#test function
c = GetTrackIDs(a)
c

['3pcCifdPTc2BbqmWpEhtUd',
 '1uRxyAup7OYrlh2SHJb80N',
 '5rIhBK9aaVMck0W2YtOwci',
 '1kBGeOp1CDUHVdbK4ergqo',
 '4CzTgOmc3Sdm4EgKQWzjQl',
 '0eZN5WsQfmNFICHuw59Zfz',
 '3LhtqibvTtjOUrzKs7Vsz1',
 '6f6pEjgfTtuRROmJ4a7Gf3',
 '4eruRiSfDY1jdT03hjyi0i',
 '3cual6JOG286qZJmCxKRAT',
 '01ZpFhrMMqKPVCwyqXneVp']

In [12]:
song_ids_album2 = []
song_ids2 = []

In [21]:
#get spotify ID's for all of the tracks in all albums
for album in tqdm(album_IDs2):
    for alb in album:
        time.sleep(0.25)
        try:
            song_ids_album2.append([GetTrackIDs(alb),alb])
            song_ids2.append([GetTrackIDs(alb)])
        except:
            song_ids_album2.append(['Spotify features Not found',alb])
            song_ids2.append(['Spotify features Not found'])        


 14%|█▎        | 3/22 [00:01<00:12,  1.47it/s]HTTP Error for GET to https://api.spotify.com/v1/albums/Spotify features Not found/tracks/ returned 400 due to invalid id

 55%|█████▍    | 12/22 [00:05<00:04,  2.47it/s]HTTP Error for GET to https://api.spotify.com/v1/albums/Spotify features Not found/tracks/ returned 400 due to invalid id

 59%|█████▉    | 13/22 [00:05<00:03,  2.54it/s]HTTP Error for GET to https://api.spotify.com/v1/albums/Spotify features Not found/tracks/ returned 400 due to invalid id

 64%|██████▎   | 14/22 [00:05<00:02,  2.71it/s]HTTP Error for GET to https://api.spotify.com/v1/albums/Spotify features Not found/tracks/ returned 400 due to invalid id

 68%|██████▊   | 15/22 [00:06<00:02,  2.61it/s]

KeyboardInterrupt: 

In [22]:
#save the song ids
with open('song_ids.pkl', 'wb') as f:
    pickle.dump(song_ids2, f)

## 4. Get Spotify data about songs

In [23]:
#function for getting information for a list of songs from song IDs
def GetTrackDescription(track_ids):
    track_descriptions = []
    for track_id in track_ids:
        description = sp.track(track_id)
        track_descriptions.append(description)
    return track_descriptions

In [24]:
#test function
d = GetTrackDescription(c)
d

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
     'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
     'id': '4Z8W4fKeB5YxbusRsdQVPb',
     'name': 'Radiohead',
     'type': 'artist',
     'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}],
   'available_markets': ['AD',
    'AE',
    'AR',
    'AT',
    'AU',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'GT',
    'HK',
    'HN',
    'HU',
    'ID',
    'IE',
    'IL',
    'IN',
    'IS',
    'IT',
    'JO',
    'JP',
    'KW',
    'LB',
    'LI',
    'LT',
    'LU',
    'LV',
    'MA',
    'MC',
    'MT',
    'MX',
    'MY',
    'NI',
    'NL',
    'NO',
    'NZ',
    'OM',
    'PA',
    'PE',
    'PH',
    'PL',
    'PS',
    '

In [25]:
#function for getting audio features for a list of tracks
def GetTrackInfo(track_ids):
    tracks_info = []
    for track_id in track_ids:
        info = sp.audio_features(track_id)
        tracks_info.append(info)
    return tracks_info

In [26]:
#test function
e = GetTrackInfo(c)
e

[[{'danceability': 0.541,
   'energy': 0.847,
   'key': 11,
   'loudness': -6.52,
   'mode': 1,
   'speechiness': 0.0297,
   'acousticness': 0.303,
   'instrumentalness': 0.271,
   'liveness': 0.109,
   'valence': 0.62,
   'tempo': 148.937,
   'type': 'audio_features',
   'id': '3pcCifdPTc2BbqmWpEhtUd',
   'uri': 'spotify:track:3pcCifdPTc2BbqmWpEhtUd',
   'track_href': 'https://api.spotify.com/v1/tracks/3pcCifdPTc2BbqmWpEhtUd',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3pcCifdPTc2BbqmWpEhtUd',
   'duration_ms': 220609,
   'time_signature': 4}],
 [{'danceability': 0.299,
   'energy': 0.263,
   'key': 9,
   'loudness': -13.207,
   'mode': 0,
   'speechiness': 0.0323,
   'acousticness': 0.968,
   'instrumentalness': 0.857,
   'liveness': 0.126,
   'valence': 0.113,
   'tempo': 137.848,
   'type': 'audio_features',
   'id': '1uRxyAup7OYrlh2SHJb80N',
   'uri': 'spotify:track:1uRxyAup7OYrlh2SHJb80N',
   'track_href': 'https://api.spotify.com/v1/tracks/1uRxyAup7OYrlh2SHJb8

In [27]:
tracks_objects = []
tracks_audio_features = []

In [28]:
#get song objects and audio features
for songs in tqdm(song_ids2):
    for song in songs:
        if song != 'Spotify features Not found':
            time.sleep(0.25)
            objects = GetTrackDescription(song)
            tracks_objects.append(objects)
            features = GetTrackInfo(song)
            tracks_audio_features.append(features)
            with open('features_spotify.pkl', 'wb') as f:
                pickle.dump(tracks_audio_features, f)
            with open('object_spotify.pkl', 'wb') as f:
                pickle.dump(tracks_objects, f)
        else:
            tracks_objects.append({'status':'Not found'})
            tracks_audio_features.append({'status':'Not found'})
            with open('features_spotify.pkl', 'wb') as f:
                pickle.dump(tracks_audio_features, f)
            with open('object_spotify.pkl', 'wb') as f:
                pickle.dump(tracks_objects, f)



  0%|          | 0/15 [00:00<?, ?it/s]

  7%|▋         | 1/15 [00:08<01:59,  8.55s/it]

 13%|█▎        | 2/15 [00:10<01:23,  6.42s/it]

 20%|██        | 3/15 [00:11<00:59,  4.96s/it]

 33%|███▎      | 5/15 [00:13<00:37,  3.71s/it]

 40%|████      | 6/15 [00:17<00:36,  4.03s/it]

 47%|████▋     | 7/15 [00:18<00:24,  3.12s/it]

 53%|█████▎    | 8/15 [00:19<00:16,  2.30s/it]

KeyboardInterrupt: 

In [37]:
tracks_audio_features[0][0]

[{'danceability': 0.714,
  'energy': 0.309,
  'key': 7,
  'loudness': -10.796,
  'mode': 1,
  'speechiness': 0.0291,
  'acousticness': 0.0157,
  'instrumentalness': 0.807,
  'liveness': 0.0777,
  'valence': 0.0671,
  'tempo': 107.346,
  'type': 'audio_features',
  'id': '7uv632EkfwYhXoqf8rhYrg',
  'uri': 'spotify:track:7uv632EkfwYhXoqf8rhYrg',
  'track_href': 'https://api.spotify.com/v1/tracks/7uv632EkfwYhXoqf8rhYrg',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7uv632EkfwYhXoqf8rhYrg',
  'duration_ms': 379533,
  'time_signature': 4}]

In [38]:
tracks_objects[0][0]

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6FXMGgJwohJLUSr5nVlf9X'},
    'href': 'https://api.spotify.com/v1/artists/6FXMGgJwohJLUSr5nVlf9X',
    'id': '6FXMGgJwohJLUSr5nVlf9X',
    'name': 'Massive Attack',
    'type': 'artist',
    'uri': 'spotify:artist:6FXMGgJwohJLUSr5nVlf9X'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',
   'PT',
   'PY',
   'QA',
   'RO',
   'SA',
   'SE',
   'SG',
   'SK

## 5. Format collected data into pandas dataframe

In [39]:
#read in pickled data
tracks_audio_features = pd.read_pickle('features_spotify.pkl')
tracks_objects = pd.read_pickle('object_spotify.pkl')

In [40]:
#flatten lists
flat_tracks_objects = [item for sublist in tracks_objects for item in sublist]
flat_tracks_audio_features = [item for sublist in tracks_audio_features for item in sublist]

In [41]:
#create dataframes with desired columns
objects_df = pd.DataFrame(columns=[
    'name',
    'duration_ms',
    'popularity',
    'num_markets',
    'album',
    'disc_number',
    'is_explicit',
    'track_number',
    'release_date',
    'artist',
    'song_id'
])

In [42]:
features_df = pd.DataFrame(columns=[
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
    'song_id'
])

In [43]:
#get song objects dictionary data into a dataframe
for song in flat_tracks_objects:
    if song == {'status': 'Not found'} or song == [None]:
        y = {
            'name': None,
            'duration_ms': None,
            'popularity': None,
            'num_markets': None,
            'album': None,
            'disc_number': None,
            'is_explicit': None,
            'track_number': None,
            'release_date': None,
            'artist': None,
            'song_id': None,
            'album_id': None
        }
        objects_df = objects_df.append(y, ignore_index=True)
    else:
        try:
            y = {
                'name': song['name'],
                'duration_ms': song['duration_ms'],
                'popularity':song['popularity'],
                'num_markets': len(song['available_markets']),
                'album': song['album']['name'],
                'disc_number': song['disc_number'],
                'is_explicit': song['explicit'],
                'track_number': song['track_number'],
                'release_date': song['album']['release_date'],
                'artist': song['artists'][0]['name'],
                'song_id': song['id'],
                'album_id': song['album']['id']
        }
        except:
             y = {
                'name': None,
                'duration_ms': None,
                'popularity': None,
                'num_markets': None,
                'album': None,
                'disc_number': None,
                'is_explicit': None,
                'track_number': None,
                'release_date': None,
                'artist': None,
                'song_id': None,
                'album_id': None
        }
        objects_df = objects_df.append(y, ignore_index=True)

In [44]:
objects_df.head()

,name,duration_ms,popularity,num_markets,album,disc_number,is_explicit,track_number,release_date,artist,song_id,album_id
0,Angel,379533,62,79,Mezzanine,1,False,1,1998-01-01,Massive Attack,7uv632EkfwYhXoqf8rhYrg,49MNmJhZQewjt06rpwp6QR
1,Risingson,298826,50,79,Mezzanine,1,False,2,1998-01-01,Massive Attack,6ggJ6MceyHGWtUg1KLp3M1,49MNmJhZQewjt06rpwp6QR
2,Teardrop,330773,69,79,Mezzanine,1,False,3,1998-01-01,Massive Attack,67Hna13dNDkZvBpTXRIaOJ,49MNmJhZQewjt06rpwp6QR
3,Inertia Creeps,357133,52,79,Mezzanine,1,False,4,1998-01-01,Massive Attack,3N2UhXZI4Gf64Ku3cCjz2g,49MNmJhZQewjt06rpwp6QR
4,Exchange,251200,47,79,Mezzanine,1,False,5,1998-01-01,Massive Attack,2HuMQkNVpFIsur2cRWWQmX,49MNmJhZQewjt06rpwp6QR


In [45]:
objects_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13916 entries, 0 to 13915
Data columns (total 12 columns):
name            13702 non-null object
duration_ms     13702 non-null object
popularity      13702 non-null object
num_markets     13702 non-null object
album           13702 non-null object
disc_number     13702 non-null object
is_explicit     13702 non-null object
track_number    13702 non-null object
release_date    13702 non-null object
artist          13702 non-null object
song_id         13702 non-null object
album_id        13702 non-null object
dtypes: object(12)
memory usage: 1.3+ MB


In [46]:
#save dataframe
with open('song_objects_df.pkl', 'wb') as f:
    pickle.dump(objects_df, f)

In [47]:
#get song features dictionary data into a dataframe
for song_object in flat_tracks_audio_features:
    for song in song_object:
        if song == {'status': 'Not found'} or song == [None]:
            y = {
                'danceability': None,
                'energy': None,
                'key': None,
                'loudness': None,
                'mode': None,
                'speechiness': None,
                'acousticness': None,
                'instrumentalness': None,
                'liveness': None,
                'valence': None,
                'tempo': None,
                'time_signature': None,
                'song_id': None
                }
            features_df = features_df.append(y, ignore_index=True)
        else:
            try:
                y = {
                    'danceability': song['danceability'],
                    'energy': song['energy'],
                    'key': song['key'],
                    'loudness': song['loudness'],
                    'mode': song['mode'],
                    'speechiness': song['speechiness'],
                    'acousticness': song['acousticness'],
                    'instrumentalness': song['instrumentalness'],
                    'liveness': song['liveness'],
                    'valence': song['valence'],
                    'tempo': song['tempo'],
                    'time_signature': song['time_signature'],
                    'song_id': song['id']
            }
            except:
                 y = {
                    'danceability': None,
                    'energy': None,
                    'key': None,
                    'loudness': None,
                    'mode': None,
                    'speechiness': None,
                    'acousticness': None,
                    'instrumentalness': None,
                    'liveness': None,
                    'valence': None,
                    'tempo': None,
                    'time_signature': None,
                    'song_id': None
            }
            features_df = features_df.append(y, ignore_index=True)

In [48]:
features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,song_id
0,0.714,0.309,7,-10.796,1,0.0291,0.0157,0.807,0.0777,0.0671,107.346,4,7uv632EkfwYhXoqf8rhYrg
1,0.521,0.892,10,-6.905,0,0.1150,0.0242,0.681,0.0667,0.2990,164.049,4,6ggJ6MceyHGWtUg1KLp3M1
2,0.615,0.419,11,-8.800,0,0.0395,0.0173,0.497,0.1350,0.2160,76.997,4,67Hna13dNDkZvBpTXRIaOJ
3,0.365,0.806,8,-8.948,1,0.0849,0.0518,0.520,0.1350,0.3880,167.383,4,3N2UhXZI4Gf64Ku3cCjz2g
4,0.705,0.210,5,-14.007,0,0.0288,0.5550,0.880,0.1110,0.4740,123.961,4,2HuMQkNVpFIsur2cRWWQmX


In [49]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14986 entries, 0 to 14985
Data columns (total 13 columns):
danceability        13702 non-null float64
energy              13702 non-null float64
key                 13702 non-null object
loudness            13702 non-null float64
mode                13702 non-null object
speechiness         13702 non-null float64
acousticness        13702 non-null float64
instrumentalness    13702 non-null float64
liveness            13702 non-null float64
valence             13702 non-null float64
tempo               13702 non-null float64
time_signature      13702 non-null object
song_id             13702 non-null object
dtypes: float64(9), object(4)
memory usage: 1.5+ MB


In [50]:
#save dataframe
# with open('song_features_df.pkl', 'wb') as f:
#     pickle.dump(features_df, f)